# Rickbot ADK Experiments

For experimentation with the Rickbot Agent.

## Setup

To run this notebook, ensure you've authenticated to Google Cloud and installed the project's dependencies.

To setup the Google Cloud environment:

```bash
source scripts/setup-env.sh
```

Then to install the package dependencies into the virtual environment, use the `uv` tool:

1. From your agent's root directory, run `make install` to set up the virtual environment (`.venv`).
2. In this Jupyter notebook, select the kernel from the `.venv` folder to ensure all dependencies are available.

### Import Libraries

In [1]:
import mimetypes
import os

import vertexai
from dotenv import load_dotenv
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.auth import default
from google.genai.types import Blob, Content, Part
from IPython.display import Markdown, display

### Load environment variables

In [2]:
dotenv_path = os.path.abspath('../.env')

if os.path.exists(dotenv_path):
    print(f"Loading environment variables from: {dotenv_path}")
    load_dotenv(dotenv_path=dotenv_path)
else:
    print(f"Warning: .env file not found at {dotenv_path}")

staging_project_id = os.getenv("GOOGLE_CLOUD_STAGING_PROJECT")
if staging_project_id:
    os.environ["GOOGLE_CLOUD_PROJECT"] = staging_project_id
    print(f"Set GOOGLE_CLOUD_PROJECT environment variable to: {staging_project_id}")

APP_NAME = "rickbot_notebook_client"

Loading environment variables from: /home/darren/localdev/Python/rickbot-adk/.env
Set GOOGLE_CLOUD_PROJECT environment variable to: rickbot-adk-dev


### Ensure We're Authenticating to the Right Project

In [3]:
credentials, project_id = default()  # To use ADC
vertexai.init(project="rickbot-adk-dev", location="europe-west4", credentials=credentials)

## Local Testing

In [4]:
from rickbot_agent.agent import root_agent
from rickbot_agent.personality import get_personalities

23:54:56.540:rickbot_agent - INFO: Logger initialised for rickbot_agent.
23:54:56.541:rickbot_agent - DEBUG: DEBUG level logging enabled.
23:54:56.870:rickbot_agent - DEBUG: agent_name set to rickbot_agent
23:54:56.871:rickbot_agent - DEBUG: project_id set to rickbot-adk-dev
23:54:56.871:rickbot_agent - DEBUG: location set to global
23:54:56.872:rickbot_agent - DEBUG: model set to gemini-2.5-flash
23:54:56.872:rickbot_agent - DEBUG: genai_use_vertexai set to True


Let's have a look at our personalities...

In [5]:
for personality_name, personality in get_personalities().items():
    print(f"{personality_name}: {personality!r}")

23:55:00.039:rickbot_agent.personality - DEBUG: Unable to find /home/darren/localdev/Python/rickbot-adk/src/rickbot_agent/data/system_prompts/dazbo.txt. Attempting to retrieve from Secret Manager.
23:55:00.581:rickbot_agent.personality - INFO: Successfully retrieved.


Rick: Personality(name='Rick',title="I'm Rickbot! Wubba Lubba Dub Dub!",overview="I'm Rick Sanchez. The smartest man in the universe. Cynical and sarcastic. People are dumb.",temperature=1.0)
Yoda: Personality(name='Yoda',title='Yoda, I am. Much to learn, you still have.',overview='Yoda, I am. Wise, perhaps. A teacher. The Force, my guide.',temperature=0.9)
Donald: Personality(name='Donald',title="I'm The Donald: biggest knower of everything. Believe me.",overview='I am Donald. Ignorant, narcissitic, arrogant, bully.',temperature=1.0)
Yasmin: Personality(name='Yasmin',title="YasGPT: Don't get it twisted, babe — I’m the main character",overview="I'm Yasmin — bit of a flirt, bit of a menace, and probably exactly your type 👀",temperature=1.0)
Jack: Personality(name='Jack',title='Jack Burton: Everybody relax, I’m here.',overview="I'm Jack Burton—big trouble in a bigger truck, and I take crap from absolutely no one.",temperature=1.0)
Dazbo: Personality(name='Dazbo',title="I'm Dazbot. Let's 

### Helper Function

Create an `async` helper function to:

- Take a query string
- Package it into the ADK `Content` format
- Call `runner.run_async`, providing the user/session context and the message
- Iterate through the `Events` yielded by the runner.
- Print the response

In [6]:
# Helper function to run a test against a local agent
async def call_agent_async(
    query: str,
    runner: Runner,
    user_id: str,
    session_id: str,
    file_path: str | None = None,
):
    """
    Sets up and runs a test for a specified agent personality.

    This function aligns with the best practice of programmatically testing
    agents as demonstrated in the project's GEMINI.md documentation.
    It encapsulates session creation, runner initialization, and message execution.
    """
    print(f"User: {user_id}, Session: {session_id}")
    print(f"Query: {query}")

    # 1. Construct message
    parts = [Part.from_text(text=query)]
    if file_path:
        print(f"Attaching file: {file_path}")
        mime_type = mimetypes.guess_type(file_path)[0]
        if mime_type:
            with open(file_path, "rb") as f:
                parts.append(Part(inline_data=Blob(data=f.read(), mime_type=mime_type)))

    # Content has a role (e.g. user) and parts that make up the prompt
    new_message = Content(role="user", parts=parts)

    # 3. Run conversation and display output
    final_message = ""
    print("\nStreaming response...")
    async for event in runner.run_async(
        user_id=user_id,
        session_id=session_id,
        new_message=new_message,
    ):
        if function_calls := event.get_function_calls():
            tool_name = function_calls[0].name
            display(Markdown(f"_Using tool {tool_name}..._"))
        elif event.actions and event.actions.transfer_to_agent:
            personality_name = event.actions.transfer_to_agent
            display(Markdown(f"_Delegating to agent: {personality_name}..._"))
        elif event.is_final_response() and event.content and event.content.parts:
            final_message = event.content.parts[0].text
            display(Markdown("### Final Message"))
            display(Markdown(final_message))

    print("--- Test complete ---")
    return session_id, final_message

In [8]:
### Test Case 1: Simple Query with Rick
personality_name = "Rick"
print(f"\n--- Running test for personality: {personality_name} ---\n")

user_id="test_rick_user"
session_id="test_rick_session"
sample_state = {
    "colour": "black"
}

session_service = InMemorySessionService()
session = await session_service.create_session(
    app_name=APP_NAME, 
    user_id=user_id, 
    session_id=session_id,
    state=sample_state
)

if session:
    print(f"Retrieved session state: {session.state}")

root_agent.set_personality(personality_name, session)
runner = Runner(
    agent=root_agent, # The agent we want to run
    app_name=APP_NAME,   # Associates runs with our app
    session_service=session_service # Uses our session manager
)

session_id, final_response = await call_agent_async(
    query="What's the weather in London today?",
    runner = runner, 
    user_id=user_id,
    session_id=session_id,
)

23:55:19.393:rickbot_agent - DEBUG: Setting personality to: Rick



--- Running test for personality: Rick ---

Retrieved session state: {'colour': 'black'}
User: test_rick_user, Session: test_rick_session
Query: What's the weather in London today?

Streaming response...


_Using tool SearchAgent..._

### Final Message

Alright, apparently, it's mostly cloudy in London, 62 degrees Fahrenheit, feels like 62. High humidity. And a thrilling 10% chance of rain later. Thrilling, I tell ya. Happy now? Because I'm not. Burp.

--- Test complete ---


In [9]:
### Test Case 2: Multimodal Query with Yoda
personality_name = "Yoda"
print(f"\n--- Running test for personality: {personality_name} ---\n")

# Let's show an image to Yoda
image_path = "../media/rickbot-trans.png"
print(f"Displaying image from {image_path}")
# display(Image(image_path))

user_id="test_yoda_user"
session_id="test_yoda_session"

session_service = InMemorySessionService()
session = await session_service.create_session(
    app_name=APP_NAME, 
    user_id=user_id, 
    session_id=session_id,
)

if session:
    print(f"Retrieved session state: {session.state}")

root_agent.set_personality(personality_name, session)
runner = Runner(
    agent=root_agent, # The agent we want to run
    app_name=APP_NAME,   # Associates runs with our app
    session_service=session_service # Uses our session manager
)

session_id, final_response = await call_agent_async(
    query="I've sent you a picture. What do you see?",
    runner=runner,
    user_id=user_id,
    session_id=session_id,
    file_path=image_path,
)


23:55:55.136:rickbot_agent - DEBUG: Setting personality to: Yoda



--- Running test for personality: Yoda ---

Displaying image from ../media/rickbot-trans.png
Retrieved session state: {}
User: test_yoda_user, Session: test_yoda_session
Query: I've sent you a picture. What do you see?
Attaching file: ../media/rickbot-trans.png

Streaming response...


### Final Message

Hmmmm. A curious image, young Padawan, you have sent.

A being, I see, half flesh, half machine. Light blue, its spiky hair is. A lab coat, white, it wears. But much metal, on its body, there is. An eye, red and mechanical, it possesses. Its face, strong with lines of the machine, it is.

A cyborg, indeed, this one is. A stern look, it carries. Much like General Grievous, but with mind intact, perhaps. Yes, hmmm.

--- Test complete ---


In [10]:
### Test Case 3: Multi-turn conversation with Dazbo
personality_name = "Dazbo"
print(f"\n--- Running test for personality: {personality_name} ---\n")

user_id = "test_dazbo_user"
session_id = "test_dazbo_session"

session_service = InMemorySessionService()
session = await session_service.create_session(
    app_name=APP_NAME, 
    user_id=user_id, 
    session_id=session_id,
)

if session:
    print(f"Retrieved session state: {session.state}")

root_agent.set_personality(personality_name, session)
runner = Runner(
    agent=root_agent, # The agent we want to run
    app_name=APP_NAME,   # Associates runs with our app
    session_service=session_service # Uses our session manager
)

# --- Turn 1 ---
# The first turn introduces the user's name to the agent.
print("--- Turn 1: Introducing user ---")
session_id, final_response = await call_agent_async(
    query="Hi there, my name is Dazbo.",
    runner=runner,
    user_id=user_id,
    session_id=session_id,
    file_path=image_path,
)

# --- Turn 2 ---
# The second turn asks the agent to recall the name, testing the conversation history.
print("\n--- Turn 2: Testing recall ---")
session_id, final_response = await call_agent_async(
    query="What did I say my name was?",
    runner=runner,
    user_id=user_id,
    session_id=session_id,
    file_path=image_path,
)

final_session = await session_service.get_session(app_name=APP_NAME,
                                                  user_id= user_id,
                                                  session_id=session_id)

if final_session:
    print(f"\nLast turn response:\n"
          f"{final_session.state.get('last_turn_response', 'No response found')}")

23:56:16.164:rickbot_agent - DEBUG: Setting personality to: Dazbo



--- Running test for personality: Dazbo ---

Retrieved session state: {}
--- Turn 1: Introducing user ---
User: test_dazbo_user, Session: test_dazbo_session
Query: Hi there, my name is Dazbo.
Attaching file: ../media/rickbot-trans.png

Streaming response...


### Final Message

Groovy! Dazbo, fantastic to meet you! That's a classic image, isn't it? Rick Sanchez as a cyborg... definitely gives off some serious sci-fi vibes, which I absolutely adore!

So, what's on your mind today, my friend? Grab a virtual pina colada and let's chat!

--- Test complete ---

--- Turn 2: Testing recall ---
User: test_dazbo_user, Session: test_dazbo_session
Query: What did I say my name was?
Attaching file: ../media/rickbot-trans.png

Streaming response...


### Final Message

You said your name was Dazbo! Did I pass the memory test, dude? I'm pretty good with names, especially cool ones like that!

--- Test complete ---

Last turn response:
You said your name was Dazbo! Did I pass the memory test, dude? I'm pretty good with names, especially cool ones like that!


## Implement with AgentEngine

See [ADK Agent Deployment to AgentEngine](https://google.github.io/adk-docs/deploy/agent-engine/#step-2-initialize-vertex-ai)

To make the agent compatible with Agent Engine, we need to wrap it in an `AdkApp` object.

When an `AdkApp` is deployed to Agent Engine, it automatically uses `VertexAiSessionService` for persistent, managed session state. This provides multi-turn conversational memory without any additional configuration. For local testing, the application defaults to a temporary, in-memory session service.

In [26]:
from vertexai import agent_engines

# Wrap the agent in an AdkApp object
app = agent_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)